In [1]:
from cmath import nan
import cv2
import os
import re
import numpy as np
from tqdm import tqdm

In [2]:
WIDTH = int(720/4)

def normilizeFrame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (WIDTH, frame.shape[0]), interpolation=cv2.INTER_NEAREST)
    return frame

cap = cv2.VideoCapture('shakalwhitenoise1min.avi')
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

ret, frame = cap.read()
if not ret:
    print('error reading frame')

gray = normilizeFrame(frame)

In [3]:
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frameCount = cap.get(cv2.CAP_PROP_FRAME_COUNT)

print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(height)
print(cap.get(cv2.CAP_PROP_POS_FRAMES))
print(frameCount)

720.0
625.0
1.0
1501.0


In [4]:
vl = gray[20]
vline = np.array([vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl])

cv2.imwrite("gray.png", vline)

True

In [5]:
#fileIn = open("out_parse.log", "rt") 
fileInBin = open("out_bin.log", "rt") 

if os.path.exists("ignored.txt"):
  os.remove("ignored.txt")

fileIgnored = open("ignored.txt", "wt")

#line = fileIn.readline()
lineBin = fileInBin.readline()


In [6]:
#falses = np.zeros(16, dtype=bool)
dictionary = []  #[frame, line]
inputLines = []
service = [] #[has pcm, bitmode(14/16)]
binaryData = []
#blockL0 = []
#blockR0 = []
#blockL1 = []
#blockR1 = []
#blockL2 = []
#blockR2 = []

In [7]:
def convertDataBlock(data):
  data = list(data.replace('#', '1').replace('-', '0'))
  data = [bool(int(numeric_string)) for numeric_string in data]
  return data

bar = tqdm(total=height * frameCount)
bar.set_description("Parsing govnologs Phase 1")
while lineBin:
    typeLine=re.match('\"?\[(V2D)\]', lineBin)
    if typeLine != None:
        typeLine = typeLine.group(1)
        if typeLine == 'V2D':
            parse = re.match('.?\[V2D\].*F\[(\d+)\].*L\[(\d+)\].*\[([#-]{137})\].*(CRC OK|No PCM!|BD CRC!)', lineBin)
            if parse and parse.group(3):
                framenum = int(parse.group(1))
                linenum = int(parse.group(2))
                currentCapPos = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                bar.update(1)
                
                #compare frame with log output
                if not framenum == currentCapPos:
                    if framenum > currentCapPos:
                        if cap.grab():
                            ret, frame = cap.retrieve()
                            
                            if not ret:
                                print('error reading frame')
                                break
                            
                            if not framenum == cap.get(cv2.CAP_PROP_POS_FRAMES):
                                print('WTF?! after read next frame number also not equals ' + str(framenum) + ' ' + str(currentCapPos))
                                break
                            
                            gray = normilizeFrame(frame)
                    else:
                        print('AHTUNG!!! ' + str(framenum) + ' ' + str(currentCapPos))
                        break
                
                dictionary.append([framenum, linenum])
                inputLines.append(gray[linenum - 1])
                binaryData.append(convertDataBlock(parse.group(3)))
                if parse.group(4) == 'No PCM!':
                    service.append([False, False])
                else:
                    service.append([True, False])
                    
    lineBin = fileInBin.readline()
cap.release()
bar.close()

Parsing govnologs Phase 1: 100%|██████████████████████████████████████████████▊| 935123/938125.0 [00:54<00:00, 17306.98it/s]


In [8]:
#def convertDataBlock(data, mode):
#  data = list(data.replace('#', '1').replace('-', '0'))
#  data = [bool(int(numeric_string)) for numeric_string in data]
#  if not mode:
#    data.insert(0, True)
#    data.insert(0, True)
#  return data
#
#offset = -1
#bar = tqdm(total= len(service))
#bar.set_description("Parsing govnologs Phase 2")
#while line:
#    typeLine=re.match('\"?\[(L2B-007)\]', line)
#    if typeLine != None:
#        typeLine = typeLine.group(1)
#        if typeLine == 'L2B-007':
#            #parse = re.match('.?\[L2B-007\].*B\[(\d+)\-(\d+)\].*(\d\d)\-BIT (?:\[|@)([#-01]{14,16})(?:\]|@) (?:\[|@)([#-01]{14,16})(?:\]|@) (?:\[|@)([#-01]{14,16})(?:\]|@) (?:\[|@)([#-01]{14,16})(?:\]|@) (?:\[|@)([#-01]{14,16})(?:\]|@) (?:\[|@)([#-01]{14,16})(?:\]|@) (?:\[|@)([#-01]{14,16})(?:\]|@) (?:\[|@)([#-01]{14,16})(?:\]|@).*(ORIG|P_FIX|Q_FIX)', line)
#            parse = re.match('.?\[L2B-007\].*B\[(\d+)\-(\d+)\].*(\d\d)\-BIT (?:\[|@)([#-01]{14})(?:\]|@) (?:\[|@)([#-01]{14})(?:\]|@) (?:\[|@)([#-01]{14})(?:\]|@) (?:\[|@)([#-01]{14})(?:\]|@) (?:\[|@)([#-01]{14})(?:\]|@) (?:\[|@)([#-01]{14})(?:\]|@)', line)
#            if parse:
#                framenum = int(parse.group(1))
#                linenum = int(parse.group(2))
#                mode = False if parse.group(3) == '14' else True
#                
#                offset = offset + 1
#                bar.update(1)
#                if not service[offset][0]: #no pcm, skip
#                    blockL0.append(falses)
#                    blockR0.append(falses)
#                    blockL1.append(falses)
#                    blockR1.append(falses)
#                    blockL2.append(falses)
#                    blockR2.append(falses)
#                    continue
#                
#                if not dictionary[offset] == [framenum, linenum]:
#                    if linenum > height:
#                        print('line ' + str(linenum) + ' not in range, skip')
#                        offset = offset - 1
#                        fileIgnored.write(line)
#                        line = fileIn.readline()
#                        continue
#                    else:
#                        print('FUCK!!! ' + str(dictionary[offset]) + ' not a ' + str([framenum, linenum]))
#                        break
#                
#                service[offset][1] = mode
#                if service[offset][0]:
#                    blockL0.append(convertDataBlock(parse.group(4), mode))
#                    blockR0.append(convertDataBlock(parse.group(6), mode))
#                    blockL1.append(convertDataBlock(parse.group(5), mode))
#                    blockR1.append(convertDataBlock(parse.group(7), mode))
#                    blockL2.append(convertDataBlock(parse.group(8), mode))
#                    blockR2.append(convertDataBlock(parse.group(9), mode))
#                else:
#                    blockL0.append(falses)
#                    blockR0.append(falses)
#                    blockL1.append(falses)
#                    blockR1.append(falses)
#                    blockL2.append(falses)
#                    blockR2.append(falses)
#                #print(str(len(dictionary)) + ' ' + str(len(inputLines)) + ' ' + str(len(service)) + ' ' + str(len(blockL0)) + ' ' + str(len(blockR0)) + ' ' + str(len(blockL1)) + ' ' + str(len(blockR1)) + ' ' + str(len(blockL2)) + ' ' + str(len(blockR2)))
#            #print(convertDataBlock(parse.group(10)))
#            #print(convertDataBlock(parse.group(11)))
#            #print(parse.group(12))  #correction
#            #break
#        else:
#            fileIgnored.write(line)
#    else:
#          fileIgnored.write(line)
#    line = fileIn.readline()
#bar.close()

In [11]:
#remove "extra" blocks
#while len(dictionary) > offset + 1:
#    del dictionary[offset]
#    del inputLines[offset]
#    del service[offset]

In [8]:
bar = tqdm(total=3)
bar.set_description("Save govnodata")

np.save('traindata/inputLines.npy', inputLines)
bar.update(1)

np.save('traindata/service.npy', service)
bar.update(1)

np.save('traindata/binaryData.npy', binaryData)
bar.update(1)

#np.save('traindata/blockL0.npy', blockL0)
#bar.update(1)
#
#np.save('traindata/blockR0.npy', blockR0)
#bar.update(1)
#
#np.save('traindata/blockL1.npy', blockL1)
#bar.update(1)
#
#np.save('traindata/blockR1.npy', blockR1)
#bar.update(1)
#
#np.save('traindata/blockL2.npy', blockL2)
#bar.update(1)
#
#np.save('traindata/blockR2.npy', blockR2)
#bar.update(1)
bar.close()

Save govnodata: 100%|█████████████████████████████████████████████████████████████████████████| 3/3 [00:11<00:00,  3.83s/it]


In [9]:
#while True:
#    if cap.grab():
#        flag, frame = cap.retrieve()
#        if not flag:
#            continue
#        else:
#            cv2.imshow('video', gray)
#    if cap.get(cv2.CAP_PROP_POS_FRAMES) == frameCount:
#        break

In [9]:
cap.release()
cv2.destroyAllWindows()